## 설치

In [ ]:
!pip install selenium
!apt-get update

# (최초 1회)
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' #
!pip install chromedriver-autoinstaller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [945 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get

In [ ]:
!python --version

import selenium
print(selenium.__version__)

Python 3.10.12
4.31.1


In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sys
from selenium.webdriver.common.keys import Keys
import urllib.request
import os
from urllib.request import urlretrieve

import time
import pandas as pd
import numpy as np
import chromedriver_autoinstaller  # setup chrome options

In [ ]:
chrome_path = "/content/drive/MyDrive/Colab Notebooks/chromedriver"

sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

chromedriver_autoinstaller.install()  # set the target URL

## 라이브러리

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import time
import openpyxl
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import re
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin
!pip3 install summa
from summa import summarizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54389 sha256=6b994fe8cf882d3a49a6fca9a4a3680ff908a9e14a25e03e0a7343b16470523f
  Stored in directory: /root/.cache/pip/wheels/4a/ca/c5/4958614cfba88ed6ceb7cb5a849f9f89f9ac49971616bc919f
Successfully built summa


# 1. 데이터 크롤링

In [ ]:
# 1. 캠퍼스픽>공모전>IT/소프트웨어/게임 링크를 main_url로 두고 게시되어 있는 각 공모전의 상세 링크를 sub_url에 저장합니다.

main_url = "https://www.campuspick.com/contest?category=108"
base_url = "https://www.campuspick.com"

driver = webdriver.Chrome(options=chrome_options)

# Chrome 웹 드라이버 설치 및 초기화
driver.get(main_url)

# 페이지가 로드될 때까지 대기 (여기서는 10초로 설정)
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.ID, 'container')))

# 스크롤을 내리면서 링크를 추가로 가져옴
sub_url = [] # url 저장 리스트
scroll_count = 0

# 초기에는 이미 로딩된 페이지의 링크들을 가져옴
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
links = soup.select("#container > div.list > div > a")
for link in links:
    absolute_url = urljoin(base_url, link.get('href'))
    sub_url.append(absolute_url)

# 페이지 맨 하단으로 내리는 코드
last_height = driver.execute_script("return document.body.scrollHeight")

while len(sub_url) < 760:
    # 페이지 맨 하단으로 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # 페이지 로딩 대기 시간 (필요에 따라 조정 가능)

    # 새로 로드된 페이지에서 링크 수집
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    new_links = soup.select("#container > div.list > div > a")

    # 새로운 링크들을 sub_url에 추가
    for link in new_links:
        absolute_url = urljoin(base_url, link.get('href'))
        sub_url.append(absolute_url)

    # 중복 링크 제거 (set을 이용하여 중복 제거 후 다시 리스트로 변환)
    sub_url = list(set(sub_url))

    # 만약 100개 이상의 링크를 수집하면 반복문 종료
    if len(sub_url) >= 100:
        break

    # 새로운 높이 계산 후 비교하여 추가 스크롤 여부 결정
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

driver.quit()

In [ ]:
# 저장한 각 공모전의 url을 팀원 수에 맞게 분배합니다.
# 각자가 하나의 리스트씩 맡아 데이터 크롤링 한 후 마지막에 하나의 데이터셋으로 합치게 됩니다.

# 리스트를 4개로 나누는 함수
def split_list(urls, n):
    # 각 부분 리스트의 크기를 계산
    k, m = divmod(len(urls), n)
    return [urls[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n)]

# 리스트를 4개로 나누기
split_urls = split_list(sub_url, 4)

# 각 리스트에 이름 부여
list1, list2, list3, list4 = split_urls

# 결과 출력
print(f"리스트 1: {list1}")
print(f"리스트 2: {list2}")
print(f"리스트 3: {list3}")
print(f"리스트 4: {list4}")

리스트 1: ['https://www.campuspick.com/contest/view?id=24989', 'https://www.campuspick.com/contest/view?id=24144', 'https://www.campuspick.com/contest/view?id=26520', 'https://www.campuspick.com/contest/view?id=25437', 'https://www.campuspick.com/contest/view?id=25249', 'https://www.campuspick.com/contest/view?id=26476', 'https://www.campuspick.com/contest/view?id=26354', 'https://www.campuspick.com/contest/view?id=25965', 'https://www.campuspick.com/contest/view?id=24128', 'https://www.campuspick.com/contest/view?id=26496', 'https://www.campuspick.com/contest/view?id=25067', 'https://www.campuspick.com/contest/view?id=26440', 'https://www.campuspick.com/contest/view?id=25555', 'https://www.campuspick.com/contest/view?id=26452', 'https://www.campuspick.com/contest/view?id=24304', 'https://www.campuspick.com/contest/view?id=26661', 'https://www.campuspick.com/contest/view?id=26491', 'https://www.campuspick.com/contest/view?id=25905', 'https://www.campuspick.com/contest/view?id=26040', 'htt

In [ ]:
# 리스트의 길이를 확인하는 코드
print(f"리스트 1의 URL 개수: {len(list1)}")
print(f"리스트 2의 URL 개수: {len(list2)}")
print(f"리스트 3의 URL 개수: {len(list3)}")
print(f"리스트 4의 URL 개수: {len(list4)}")

리스트 1의 URL 개수: 26
리스트 2의 URL 개수: 25
리스트 3의 URL 개수: 25
리스트 4의 URL 개수: 25


In [ ]:
# 2. 크롤링 함수 만들기
# 1) 각자 맡은 list에 저장된 웹페이지 해당 주소로 이동하여 크롤링 함수 실행
# 2) 크롤링 함수
# 공모전 상세 페이지에서 공모전명, 주최기관명, 기간, 상금, 공모전 소개를 가져옵니다.
# 요약 칼럼의 경우 공모전 소개의 내용을 30% 요약하는 코드의 반환값입니다.
# 위의 데이터를 엑셀에 계속 추가하는 함수를 만듭니다.

### list1, list2, list3, list4 순서대로 넣기

In [ ]:
def scrape_contest_info():
    results = []
    driver = webdriver.Chrome(options=chrome_options)

    for i in range(len(list1)): # 자신이 맡은 리스트에 저장된 sub_url 개수
        # 웹페이지 해당 주소로 이동
        driver.get(list1[i]) # 자신이 맡은 리스트
        time.sleep(3)
        wait = WebDriverWait(driver, 20)

         # 공모전명
        try:
            c_name = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'h1'))).text
            print(c_name)
        except:
            c_name = "없음"

        # 주최기관명
        try:
            c_company = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'p.company'))).text
            print(c_company)
        except:
            c_company = "없음"

        # 기간
        try:
            due_date = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'p.indent'))).text
            print(due_date)
        except:
            due_date = "없음"

        # 상금
        try:
            # 1등 시상금 추출
            first_prize = driver.find_element(By.XPATH, "//h3[text()='1등시상금']/following-sibling::p[@class='indent']").text
        except Exception as e:
            print(f"1등 시상금 크롤링 실패: {e}")
            first_prize = "정보 없음"

        try:
            # 총 시상금 추출
            total_prize = driver.find_element(By.XPATH, "//h3[text()='총시상금']/following-sibling::p[@class='indent']").text
        except Exception as e:
            print(f"총 시상금 크롤링 실패: {e}")
            total_prize = "정보 없음"

        # 상금 정보 결합
        prize_money = f"1등 시상금: {first_prize}, 총 시상금: {total_prize}"

        # 공모전 소개
        try:
            description = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'article.description'))).text
            print(description)
        except:
            description = "없음"


        # 요약 정보
        try:
          # description이 "없음"이 아닌 경우에만 요약 생성
          if description and description != "없음":
             summary = summarizer.summarize(description, ratio=0.3)  # 원문의 약 30%만을 요약하여 반환
          else:
              summary = '없음'
        except Exception as e:
            print(f"요약 생성 실패: {e}")
            summary = '없음'

        # 결과에 정보 저장
        result = {'공모전명': c_name, '기관명': c_company,'기간':due_date ,'상금': prize_money, '공모전 소개': description, '요약': summary}
        results.append(result)

    return pd.DataFrame(results)

In [ ]:
# 웹크롤링 결과를 확인하고 데이터셋을 csv 파일로 저장합니다.
results1 = scrape_contest_info()
results1

2024 월드퀀트 국제 퀀트 모의투자 대회(IQC)
월드퀀트
3월 19일(화) ~ 5월 15일(수)
1등 시상금 크롤링 실패: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//h3[text()='1등시상금']/following-sibling::p[@class='indent']"}
  (Session info: chrome-headless-shell=127.0.6533.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5a403fe006ca <unknown>
#1 0x5a403fad1600 <unknown>
#2 0x5a403fb20bcb <unknown>
#3 0x5a403fb20eb1 <unknown>
#4 0x5a403fb64b24 <unknown>
#5 0x5a403fb438cd <unknown>
#6 0x5a403fb6204a <unknown>
#7 0x5a403fb43643 <unknown>
#8 0x5a403fb13d31 <unknown>
#9 0x5a403fb1479e <unknown>
#10 0x5a403fdc825b <unknown>
#11 0x5a403fdcc1f2 <unknown>
#12 0x5a403fdb5615 <unknown>
#13 0x5a403fdccd82 <unknown>
#14 0x5a403fd9a25f <unknown>
#15 0x5a403fdefe68 <unknown>
#16 0x5a403fdf0040 <unknown>
#17 0x5a403fdff49c <unknown>
#18 0x7e10ce0e8ac3 <unknown>

2024 월드퀀트 국제 퀀트 모의투자(IQC) 대회




,공모전명,기관명,기간,상금,공모전 소개,요약
0,2024 월드퀀트 국제 퀀트 모의투자 대회(IQC),월드퀀트,3월 19일(화) ~ 5월 15일(수),"1등 시상금: 정보 없음, 총 시상금: 5천만원이상",2024 월드퀀트 국제 퀀트 모의투자(IQC) 대회\n\n\n\n○ 공모개요\n- ...,2024 월드퀀트 국제 퀀트 모의투자(IQC) 대회\n○ 공모개요\n- Intern...
1,제4회 ICT융‧복합 기반 치의학산업 사업화 전국 아이디어 경진대회,(재)부산테크노파크,2023년 10월 16일(월) ~ 2023년 12월 10일(일),"1등 시상금: 3,000,000원, 총 시상금: 1천만원이하",* 모집기간 연장 (~12.10)\n[(재)부산테크노파크] 제4회 ICT융‧복합 기...,"- 의료산업 종사, (예비) 창업자 및 대학(원)생 등 ICT 치의학의료기기 산업에..."
2,(추가모집) 2024 AI 헬스케어 아이디어톤 경진대회,강원지역혁신플랫폼,8월 8일(목) ~ 8월 11일(일),"1등 시상금: 1,500,000원, 총 시상금: 1천만원이하","● 참가 자격\n- 강원지역혁신플랫폼 참여대학 대학(원)생\n* 가톨릭관동대학교, ...",● 참가 자격\n● 대회 주제 : IT 기술과 강원도 특화 산업을 융합한 AI 헬스...
3,[영림원소프트랩] 2024년 춘계통합학술대회 대학생 ERP 아이디어 공모,(주)영림원소프트랩,4월 11일(목) ~ 5월 7일(화),"1등 시상금: 2,000,000원, 총 시상금: 정보 없음",(주)영림원소프트랩과 한국경영정보학회 공동 주관으로 2024년 춘계통합학술대회에서 ...,- 5/7(화) 오전 9시까지 아이디어 개요서 제출 마감\n(상세 예시 첨부 포스터...
4,2024 핵테온 세종 국제 사이버 보안 위크 안내,세종특별자치시,3월 25일(월) ~ 4월 21일(일),"1등 시상금: 10,000,000원, 총 시상금: 5천만원~3천만원",세종특별자치시에서는 대학생 사이버보안 역량 강화 및 인재 육성·발굴·지원을 위해 2...,「핵테온 세종(HackTheon* Sejong)」 국제 대학생 사이버보안 경진대회를...
5,2024년 GovTech 창업 경진대회(총 상금 8천만 원) 모집 안내,"디지털플랫폼정부위원회, 과학기술정보통신부",8월 5일(월) ~ 8월 30일(금),"1등 시상금: 15,000,000원, 총 시상금: 5천만원이상",『2024년 GovTech 창업 경진대회』\n\n1. 경진대회 개요\n\n▶ 개요\...,▶ 개요\n▶ 추진절차\n▶ 참가자격\n▶ 분야\n- 아이디어 기획 분야(자유공모)...
6,2024년 AI+Security 아이디어 공모전 개최,과학기술정보통신부,7월 12일(금) ~ 8월 12일(월),"1등 시상금: 3,000,000원, 총 시상금: 1천만원이하","■ 대상\n대한민국 국민 누구나\n※ 개인 또는 단체(4인 이하)로 참여 가능, 1...",■ 대상\nAI를 활용하여 보안 또는 안전을 강화할 수 있는 서비스 아이디어 제안\...
7,"상상을 크게, 개발은 빠르게! 로우코드 앱 개발 공모전","영림원소프트랩, 플렉스튜디오, ZDnet",6월 10일(월) ~ 9월 20일(금),"1등 시상금: 10,000,000원, 총 시상금: 5천만원~3천만원",■ 공모 주제\n#누군가의 일상을 더 편리하게 만드는 앱\n\n가상의 사용자를 정하...,■ 공모 주제\n■ 공모 내용\n■ 공모 기간\n■ 지원 방법\n- https://...
8,공간융합 빅데이터 활용 아이디어 기획 및 데이터 분석·시각화 공모전,(주)제타럭스시스템,2023년 11월 4일(토) ~ 2023년 11월 26일(일),"1등 시상금: 1,000,000원, 총 시상금: 1천만원이하",[(주)제타럭스시스템]공간융합 빅데이터 활용 아이디어 기획 및 데이터 분석·시각화 ...,[(주)제타럭스시스템]공간융합 빅데이터 활용 아이디어 기획 및 데이터 분석·시각화 ...
9,제3회 게임제너레이션 게임비평공모전,게임문화재단,7월 25일(목) ~ 9월 7일(토),"1등 시상금: 1,200,000원, 총 시상금: 1천만원이하",제3회 게임제너레이션 게임비평공모전 안내\n\n\n게임제너레이션은 한국 디지털게임 ...,■ 공모형식 및 참여방법:\n기존 GG 아티클 및 공모전 수상작 참조)\n■ 시상내...


In [ ]:
def scrape_contest_info():
    results = []
    driver = webdriver.Chrome(options=chrome_options)

    for i in range(len(list2)): # 자신이 맡은 리스트에 저장된 sub_url 개수
        # 웹페이지 해당 주소로 이동
        driver.get(list2[i]) # 자신이 맡은 리스트
        time.sleep(3)
        wait = WebDriverWait(driver, 20)

         # 공모전명
        try:
            c_name = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'h1'))).text
            print(c_name)
        except:
            c_name = "없음"

        # 주최기관명
        try:
            c_company = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'p.company'))).text
            print(c_company)
        except:
            c_company = "없음"

        # 기간
        try:
            due_date = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'p.indent'))).text
            print(due_date)
        except:
            due_date = "없음"

        # 상금
        try:
            # 1등 시상금 추출
            first_prize = driver.find_element(By.XPATH, "//h3[text()='1등시상금']/following-sibling::p[@class='indent']").text
        except Exception as e:
            print(f"1등 시상금 크롤링 실패: {e}")
            first_prize = "정보 없음"

        try:
            # 총 시상금 추출
            total_prize = driver.find_element(By.XPATH, "//h3[text()='총시상금']/following-sibling::p[@class='indent']").text
        except Exception as e:
            print(f"총 시상금 크롤링 실패: {e}")
            total_prize = "정보 없음"

        # 상금 정보 결합
        prize_money = f"1등 시상금: {first_prize}, 총 시상금: {total_prize}"

        # 공모전 소개
        try:
            description = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'article.description'))).text
            print(description)
        except:
            description = "없음"


        # 요약 정보
        try:
          # description이 "없음"이 아닌 경우에만 요약 생성
          if description and description != "없음":
             summary = summarizer.summarize(description, ratio=0.3)  # 원문의 약 30%만을 요약하여 반환
          else:
              summary = '없음'
        except Exception as e:
            print(f"요약 생성 실패: {e}")
            summary = '없음'

        # 결과에 정보 저장
        result = {'공모전명': c_name, '기관명': c_company,'기간':due_date ,'상금': prize_money, '공모전 소개': description, '요약': summary}
        results.append(result)

    return pd.DataFrame(results)

In [ ]:
results2 = scrape_contest_info()
results2

제2회 스마트축산 AI 경진대회 공고
축산물품질평가원
5월 23일(목) ~ 7월 23일(화)
제2회 스마트축산 AI 경진대회 공고





○ 공모개요

-제2회 스마트축산 AI 경진대회



○ 공모주제

-AI·빅데이터 기술 활용 축산 현장문제 해결



○ 기간 및 일정

-2024. 5. 23. ~ 7. 23.



○ 지원자격

-스마트축산 AI․솔루션 기술 보유 기업, 스마트축산 관련 대학 및 일반대학

* (상용화 기술 사례) 축산분야 AI기술 보유기관(기업, 단체 등), IT․ICT전문가 등으로 팀 구성

** (알고리즘 개발) 스마트축산 데이터 알고리즘 개발 가능한 ① 기업, 단체, ② 대학생 등 팀 구성

*** 축종별 복수 주제 신청 가능




○ 접수방법

-담당자 메일 제출(smart_livestock@ekape.or.kr)




○ 제출서류

- 상용화 기술 사례: 참가신청서, 상용화 기술 사례 제안서, 제안 요약서, 정보 확인 및 조회 동의서, 홍보 활용 동의서

- 알고리즘 개발: 참가신청서, 알고리즘 개발 제안서, 제안 요약서, 정보 확인 및 조회 동의서, 홍보 활용 동의서




○ 심사기준

-1차 서면심사, 2차 전문가 검정(제안서 검증), 시상식 당일 제안발표 및 현장 투표 결과를 통한 최종 수상팀(8개 팀) 선발





○ 시상내역

-총상금 2,500만원 (8점)

기업, 단체 등

우수기술 대상(1점) 500만원, 최우수상(1점) 300만원, 우수상(1점) 200만원

알고리즘 개발 대상(1점) 500만원, 최우수상(1점) 300만원, 우수상(1점) 200만원



대학생

알고리즘 개발 대상(1점) 300만원, 최우수상(1점) 200만원




○ 문의

- ☏축산물품질평가원 스마트축산지원단(044-410-7303~5) 및 콜센터(1555-5927)
16회 코드게이트 글로벌 AI아이디어랩 공모전
사단법인코드게이트보안포럼, 금융보안원
4월 29일(월) ~ 6월 30일(일)
● 참가 자격
- 제한 없음. AI에 관

,공모전명,기관명,기간,상금,공모전 소개,요약
0,제2회 스마트축산 AI 경진대회 공고,축산물품질평가원,5월 23일(목) ~ 7월 23일(화),"1등 시상금: 5,000,000원, 총 시상금: 3천만원~1천만원",제2회 스마트축산 AI 경진대회 공고\n\n\n\n\n\n○ 공모개요\n\n-제2회...,"○ 기간 및 일정\n-스마트축산 AI․솔루션 기술 보유 기업, 스마트축산 관련 대학..."
1,16회 코드게이트 글로벌 AI아이디어랩 공모전,"사단법인코드게이트보안포럼, 금융보안원",4월 29일(월) ~ 6월 30일(일),"1등 시상금: 20,000,000원, 총 시상금: 5천만원~3천만원",● 참가 자격\n- 제한 없음. AI에 관심 있는 누구나\n* 접수일 기준 만 14...,- 보안 문제 해결을 위한 AI 활용 기술 또는 서비스 아이디어\n- 1위 : 1팀...
2,2024 제1회 대한민국 전통조경대전 (디지털 설계공모),국가유산청,6월 17일(월) ~ 9월 25일(수),"1등 시상금: 10,000,000원, 총 시상금: 3천만원~1천만원","■ 주제 : 디지털로 만나는 선비의 이상향, 별서정원\n\n■ 공모 대상\n① 전통...","■ 공모 대상\n② 관련 문헌, 회화, 지도, 시 등에 숨겨진 전통조경 원리를 찾아..."
3,[부천국제애니메이션페스티벌] BIAF2024 애니프리젠테이션 공모,부천국제애니메이션페스티벌,8월 1일(목) ~ 9월 23일(월),"1등 시상금: 1,600,000원, 총 시상금: 1천만원이하",📆 접수기간\n- 08. 01.(목) ~ 09. 23.(월) 17:00까지\n\n✏...,"- 진학, 졸업, 취업, 창업을 목표로 하는 대한민국 국적 소지자 누구나 (나이, ..."
4,디지털 심화쟁점 논문 공모전,과학기술정보통신부,7월 17일(수) ~ 9월 19일(목),"1등 시상금: 3,000,000원, 총 시상금: 1천만원이하",◉ 과학기술정보통신부 공고 제2024-0781호\n과학기술정보통신부(주최)와 한국정...,과학기술정보통신부 장관
5,2024 RE-DESIGN THON(메이커톤),서울과학기술대학교 창업지원단,5월 20일(월) ~ 6월 2일(일),"1등 시상금: 2,000,000원, 총 시상금: 1천만원이하",■ 공모 주제\n\n: **2024 RE-DESIGN THON(메이커톤)**\n\n...,■ 공모 주제\n■ 공모 내용\n■ 지원 자격\n※ 지원 대상자 중 예선(서면평가)...
6,제2회 한올바이오파마 약대생 아이디어 공모전,한올바이오파마,2023년 12월 26일(화) ~ 3월 3일(일),"1등 시상금: 정보 없음, 총 시상금: 정보 없음",[한올바이오파마] 제2회 한올바이오파마 약대생 아이디어 공모전\n[바로가기 >>> ...,"■ 공모주제\n- ""AI를 활용한 신약개발 아이디어 및 전략 제안\n한올바이오파마가..."
7,2024년 광진구 빅데이터 분석 공모전,서울특별시 광진구,4월 1일(월) ~ 5월 3일(금),"1등 시상금: 3,000,000원, 총 시상금: 1천만원이하",○ 공모주제 : 광진구 현안 관련 자유주제\n※ 2023년과 달리 주제에 따른 가점...,○ 응모조건 : 광진구에 관심있는 전 국민(개인 또는 팀 구성하여 참가 가능)\n○...
8,디지털 심화쟁점 토론대회(기한연장),과학기술정보통신부,7월 17일(수) ~ 8월 25일(일),"1등 시상금: 3,000,000원, 총 시상금: 3천만원~1천만원",● 참가 자격\n- 중등부 : 중학교에 재학중이거나 2009년생부터 2011년생에 ...,- 중등부 : 중학교에 재학중이거나 2009년생부터 2011년생에 해당하는 자\n-...
9,제2회 개인정보보호 모의재판 경연대회,개인정보보호위원회,6월 12일(수) ~ 7월 17일(수),"1등 시상금: 3,000,000원, 총 시상금: 3천만원~1천만원",개인정보보호위원회에서 <제2회 개인정보보호 모의재판 경연대회>를 개최합니다.\n\n...,개인정보보호위원회에서 <제2회 개인정보보호 모의재판 경연대회>를 개최합니다.\n개인...


In [ ]:
def scrape_contest_info():
    results = []
    driver = webdriver.Chrome(options=chrome_options)

    for i in range(len(list3)): # 자신이 맡은 리스트에 저장된 sub_url 개수
        # 웹페이지 해당 주소로 이동
        driver.get(list3[i]) # 자신이 맡은 리스트
        time.sleep(3)
        wait = WebDriverWait(driver, 20)

         # 공모전명
        try:
            c_name = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'h1'))).text
            print(c_name)
        except:
            c_name = "없음"

        # 주최기관명
        try:
            c_company = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'p.company'))).text
            print(c_company)
        except:
            c_company = "없음"

        # 기간
        try:
            due_date = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'p.indent'))).text
            print(due_date)
        except:
            due_date = "없음"

        # 상금
        try:
            # 1등 시상금 추출
            first_prize = driver.find_element(By.XPATH, "//h3[text()='1등시상금']/following-sibling::p[@class='indent']").text
        except Exception as e:
            print(f"1등 시상금 크롤링 실패: {e}")
            first_prize = "정보 없음"

        try:
            # 총 시상금 추출
            total_prize = driver.find_element(By.XPATH, "//h3[text()='총시상금']/following-sibling::p[@class='indent']").text
        except Exception as e:
            print(f"총 시상금 크롤링 실패: {e}")
            total_prize = "정보 없음"

        # 상금 정보 결합
        prize_money = f"1등 시상금: {first_prize}, 총 시상금: {total_prize}"

        # 공모전 소개
        try:
            description = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'article.description'))).text
            print(description)
        except:
            description = "없음"


        # 요약 정보
        try:
          # description이 "없음"이 아닌 경우에만 요약 생성
          if description and description != "없음":
             summary = summarizer.summarize(description, ratio=0.3)  # 원문의 약 30%만을 요약하여 반환
          else:
              summary = '없음'
        except Exception as e:
            print(f"요약 생성 실패: {e}")
            summary = '없음'

        # 결과에 정보 저장
        result = {'공모전명': c_name, '기관명': c_company,'기간':due_date ,'상금': prize_money, '공모전 소개': description, '요약': summary}
        results.append(result)

    return pd.DataFrame(results)

In [ ]:
results3 = scrape_contest_info()
results3

노원구 청년 빅데이터 활용 아이디어 공모전
노원구 청년정책과
2023년 11월 27일(월) ~ 2023년 12월 3일(일)
[노원구 청년정책과] 노원구 청년 빅데이터 활용 아이디어 공모전
[바로가기 >>> https://bit.ly/47YAnWV]

■ 참가자격

- 19세 이상 ~ 39세 이하 노원구 거주 또는 재직/재학 등 활동 청년 1인 이상을 포함한 팀(또는 개인)

■ 공모주제

- 노원구 데이터를 잘 활용할 수 있는 아이디어로 아래의 2개 중에서 1개 선택

· ① 노원구 청년을 위한 청년정책
(일자리, 주거, 복지, 금융, 문화예술, 교육, 참여 등)

· ② 노원구민을 위한 정책 (청년들을 위한 정책 이외)

■ 시상내역

- 총 상금 290만원 규모 (6팀 시상)

■ 일정

- 모집기간: 2023년 11월 27일 ~ 12월 3일

- 1차(서면) 심사 결과 발표(제출 보고서 기반 평가) : 2023년 12월 8일

- 2차(현장) 심사 : 2023년 12월 14일 (목) 오후 2시, 노원구청
(1차 심사 통과 팀 대상)

■ 지원방법

- 홈페이지 참조

■ 문의

[ 한국경영인증원 미래전략팀 ]

- ☏ 02-6309-9005

- 이메일 문의: hjkim@ikmr.co.kr
2024 MATLAB 대학생 AI 경진대회
매스웍스코리아
4월 25일(목) ~ 6월 20일(목)
▪ 공모주제
- 인공지능 기술을 우리 실생활에 새롭게 적용하거나 산업에서 기존의 프로세스를 개선할 수 있는 아이디어를 MATLAB으로 구현
*MATLAB 및 MATLAB toolbox 활용 필수

▪ 기간 및 일정
- 예선: 2024. 6. 20(목) 오후 6시 참가 신청 접수 마감
- 본선: 2024. 7. 31(수) 오후 6시 영상 접수 마감
- 수상자 발표: 2024. 9. 25(수)

▪ 지원자격
- 대학교 재학 중인 학부생 또는 대학원생 (개인 또는 팀)

▪ 접수방법 및 심사기준
- 대회 홈페이지 참고 (https://bit.ly/44fnres)

▪ 제출내용
-

,공모전명,기관명,기간,상금,공모전 소개,요약
0,노원구 청년 빅데이터 활용 아이디어 공모전,노원구 청년정책과,2023년 11월 27일(월) ~ 2023년 12월 3일(일),"1등 시상금: 1,000,000원, 총 시상금: 1천만원이하",[노원구 청년정책과] 노원구 청년 빅데이터 활용 아이디어 공모전\n[바로가기 >>>...,- 19세 이상 ~ 39세 이하 노원구 거주 또는 재직/재학 등 활동 청년 1인 이...
1,2024 MATLAB 대학생 AI 경진대회,매스웍스코리아,4월 25일(목) ~ 6월 20일(목),"1등 시상금: 2,000,000원, 총 시상금: 1천만원이하",▪ 공모주제\n- 인공지능 기술을 우리 실생활에 새롭게 적용하거나 산업에서 기존의 ...,▪ 공모주제\n▪ 기간 및 일정\n7. 31(수) 오후 6시 영상 접수 마감\n▪ ...
2,LG에너지솔루션 BSS 마케팅/기술 아이디어 공모전,LG에너지솔루션,6월 1일(토) ~ 7월 31일(수),"1등 시상금: 3,000,000원, 총 시상금: 1천만원이하",■ 공모 주제\n\nLG에너지솔루션 BSS 마케팅/기술 아이디어 공모전\n\n\n\...,LG에너지솔루션 BSS 마케팅/기술 아이디어 공모전\n-마케팅 분야: 대학생 및 일...
3,세중그룹 물류기술 공모전,세중그룹,5월 27일(월) ~ 6월 30일(일),"1등 시상금: 1,000,000원, 총 시상금: 1천만원이하",[공모주제] (택1)\n\n** E-Commerce 부문 (택1) **\n1. 해외...,1. 해외 글로벌 플랫폼을 통한 주소 및 상품정보를 AI번역을 통해 정확도와 처리속...
4,2024 의료 인공지능 아이디어 경진대회,보건복지부,7월 3일(수) ~ 8월 2일(금),"1등 시상금: 5,000,000원, 총 시상금: 3천만원~1천만원",◈ 주최 / 주관 ◈\n▷ 주최 : 보건복지부\n▷ 주관 : 한국보건산업진흥원\n\...,▷ 의료 분야 인공지능 기술 적용을 위한 솔루션 아이디어 구현 및 맞춤형 모델 개발...
5,[SCPC 2024] 제10회 삼성전자 대학생 프로그래밍 경진대회,삼성리서치,6월 4일(화) ~ 7월 4일(목),"1등 시상금: 20,000,000원, 총 시상금: 5천만원이상",[SCPC 2024] 제10회 삼성전자 대학생 프로그래밍 경진대회\n\n• 공모개요...,[SCPC 2024] 제10회 삼성전자 대학생 프로그래밍 경진대회\n• 공모개요\n...
6,생성형 AI 활용 캐릭터 디자인 공모전,엑스포디자인그룹,2월 21일(수) ~ 3월 21일(목),"1등 시상금: 1,000,000원, 총 시상금: 1천만원이하",[엑스포디자인그룹] 생성형 AI 활용 캐릭터 디자인 공모전\n[바로가기 >>> 하단...,[엑스포디자인그룹] 생성형 AI 활용 캐릭터 디자인 공모전\n- 엑스포디자인그룹의 ...
7,[국방부·병무청·방위사업청] 2024년 국방 공공데이터 활용 경진대회,국방부·병무청·방위사업청,4월 22일(월) ~ 6월 10일(월),"1등 시상금: 3,000,000원, 총 시상금: 3천만원~1천만원","○ 공모개요\n국방 공공데이터 활용 확산을 추진하고, 공공데이터를 활용한 아이디어 ...",- 대상 : 국방부장관 상장 및 상금 150만원\n- 최우수상 : 병무청장 상장 및...
8,제 4회 소셜임팩트 디지털아트 경연대회,한국조지메이슨대학교,2월 5일(월) ~ 4월 14일(일),"1등 시상금: 1,000,000원, 총 시상금: 1천만원이하",주제: 우리 사회가 직면한 다양한 문제에 컴퓨터 게임 및 디지털미디어를 활용해 대중...,주제: 우리 사회가 직면한 다양한 문제에 컴퓨터 게임 및 디지털미디어를 활용해 대중...
9,제 6회 KB국민은행 AI챌린지 미래 금융 경진대회,KB국민은행,7월 1일(월) ~ 8월 11일(일),"1등 시상금: 10,240,000원, 총 시상금: 3천만원~1천만원",FUTURE FINANCE\n제 6회 A.I.기술을 활용한 미래금융 경진대회\nA....,[주제] : AI기술을 활용한 금융관련 서비스 및 아이디어(① ~ ② 중 택1)\n...


In [ ]:
def scrape_contest_info():
    results = []
    driver = webdriver.Chrome(options=chrome_options)

    for i in range(len(list4)): # 자신이 맡은 리스트에 저장된 sub_url 개수
        # 웹페이지 해당 주소로 이동
        driver.get(list4[i]) # 자신이 맡은 리스트
        time.sleep(3)
        wait = WebDriverWait(driver, 20)

         # 공모전명
        try:
            c_name = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'h1'))).text
            print(c_name)
        except:
            c_name = "없음"

        # 주최기관명
        try:
            c_company = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'p.company'))).text
            print(c_company)
        except:
            c_company = "없음"

        # 기간
        try:
            due_date = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'p.indent'))).text
            print(due_date)
        except:
            due_date = "없음"

        # 상금
        try:
            # 1등 시상금 추출
            first_prize = driver.find_element(By.XPATH, "//h3[text()='1등시상금']/following-sibling::p[@class='indent']").text
        except Exception as e:
            print(f"1등 시상금 크롤링 실패: {e}")
            first_prize = "정보 없음"

        try:
            # 총 시상금 추출
            total_prize = driver.find_element(By.XPATH, "//h3[text()='총시상금']/following-sibling::p[@class='indent']").text
        except Exception as e:
            print(f"총 시상금 크롤링 실패: {e}")
            total_prize = "정보 없음"

        # 상금 정보 결합
        prize_money = f"1등 시상금: {first_prize}, 총 시상금: {total_prize}"

        # 공모전 소개
        try:
            description = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'article.description'))).text
            print(description)
        except:
            description = "없음"


        # 요약 정보
        try:
          # description이 "없음"이 아닌 경우에만 요약 생성
          if description and description != "없음":
             summary = summarizer.summarize(description, ratio=0.3)  # 원문의 약 30%만을 요약하여 반환
          else:
              summary = '없음'
        except Exception as e:
            print(f"요약 생성 실패: {e}")
            summary = '없음'

        # 결과에 정보 저장
        result = {'공모전명': c_name, '기관명': c_company,'기간':due_date ,'상금': prize_money, '공모전 소개': description, '요약': summary}
        results.append(result)

    return pd.DataFrame(results)

In [ ]:
results4 = scrape_contest_info()
results4

2024 스마일게이트 인디게임 창작 공모전 <IndieGo>
스마일게이트 퓨처랩
2월 5일(월) ~ 3월 5일(화)
2024 스마일게이트 인디게임 창작 공모전 IndieGo

○ 공모개요

- IndieGo 2024 스마일게이트 인디게임 창작 공모전입니다.

○ 공모주제

- 자유(제한없음)

○ 기간 및 일정

- 공모기간: 2024년 2월 5일(월) - 3월 5일(화) 오후 6시
- 결과발표: 2024년 4월 8일(월)
- 밋업 및 시상식: 2024년 4월 15일(월)

○ 지원자격

- PC/모바일 인디게임을 개발하고 있는 창작팀 (1인 창작자 가능)
- 공고일 기준 출시하지 않은 게임 (데모 공개 가능, 얼리억세스/정식 출시 불가능)

※ 스마일게이트멤버십 인디게임 부문 수료작, 이전 IndieGo 수상작은 지원 불가능합니다.

○ 접수방법

- 스마일게이트 퓨처랩을 통한 온라인 지원

○ 제출서류

- 실행 가능한 프로토타입(exe 혹은 안드로이드 APK)이 담긴 구글 드라이브 링크 제출
- 1분 내외 게임 소개영상(실제 플레이 장면 포함)
- 게임 기획서(자율 양식, 분량 제한없음)

※ 구글 드라이브 링크 제출 시 반드시 권한 부여를 부탁드립니다.
(다운로드 불가 시 심사 불가)
※ 출시 혹은 데모버전을 공개한 경우에도 꼭 빌드 파일을 제출해 주세요.
(페이지 주소 등 링크 제출 불가)
※ 프로토타입에 오류가 발생하는 경우 사전 통보 없이
심사에 불이익이 있을 수 있습니다.
※ 심사 및 실행 시, PC 및 스마트폰 외 별도 하드웨어(VR 헤드셋 등)가
필요한 경우 심사가 불가합니다.

○ 시상내역

- 대상(1팀) : 상장 및 상금 500만원
- 최우수상(2팀) : 상장 및 상금 300만원
- 장려상(4팀) : 상장 및 상금 100만원

○ 유의사항

- 응모 콘텐츠의 저작권은 해당 창작자(팀)에게 있습니다. 단, 수상작의 경우
스마일게이트 희망스튜디오는 비영리 목적으로 창작 콘텐츠를 공유하거나
활용할 수 있습니다. (홈페이지 게시, 영상 콘텐츠 

,공모전명,기관명,기간,상금,공모전 소개,요약
0,2024 스마일게이트 인디게임 창작 공모전 <IndieGo>,스마일게이트 퓨처랩,2월 5일(월) ~ 3월 5일(화),"1등 시상금: 5,000,000원, 총 시상금: 3천만원~1천만원",2024 스마일게이트 인디게임 창작 공모전 IndieGo\n\n○ 공모개요\n\n-...,2024 스마일게이트 인디게임 창작 공모전 IndieGo\n- IndieGo 202...
1,17회 알테어 최적화 대회(AOC 2024),알테어,5월 1일(수) ~ 5월 31일(금),"1등 시상금: 2,000,000원, 총 시상금: 1천만원이하",자유주제 - 알테어 솔루션을 활용하여 최적 설계가 된 제품 개발\n\n참가 자격 -...,본선 진출자 발표 8.8\n본선 8.14
2,제 7회 핀테크 아이디어 공모전,금융위원회,5월 20일(월) ~ 7월 10일(수),"1등 시상금: 20,000,000원, 총 시상금: 5천만원이상",<제 7회 핀테크 아이디어 공모전>\n\n접수기간 : 2024.5.20. (월) ~...,"공모주제 : 핀테크 관련 비즈니스 모델, 사업 및 기술아이디어 등\n- 대상 (1개..."
3,AI 인공지능 활용 아이디어 공모전,재단법인 미래와소프트웨어,6월 10일(월) ~ 8월 11일(일),"1등 시상금: 5,000,000원, 총 시상금: 3천만원~1천만원",AI 인공지능 활용 아이디어 (공모전)\n[재단법인 미래와소프트웨어] AI 인공지능...,AI 인공지능 활용 아이디어 (공모전)\n[재단법인 미래와소프트웨어] AI 인공지능...
4,개방형 클라우드 플랫폼(K-PaaS) 기반 서비스 개발 아이디어 공모전,과학기술정보통신부,5월 31일(금) ~ 10월 31일(목),"1등 시상금: 5,000,000원, 총 시상금: 5천만원~3천만원",제8회 개방형 클라우드 플랫폼(K-PaaS) 기반 서비스 개발 아이디어 공모전\n\...,제8회 개방형 클라우드 플랫폼(K-PaaS) 기반 서비스 개발 아이디어 공모전\n-...
5,[농림축산식품부] 스마트농업 AI 경진대회,농림축산식품부,7월 2일(화) ~ 7월 26일(금),"1등 시상금: 50,000,000원, 총 시상금: 5천만원이상",스마트농업 AI 경진대회 참가 모집\n\n\n\n○ 공모개요\n\n- 인공지능(AI...,스마트농업 AI 경진대회 참가 모집\n○ 공모주제\n○ 기간 및 일정\n○ 지원자격...
6,제4회 경상북도 공공데이터 활용 창업경진대회,경상북도,5월 20일(월) ~ 6월 21일(금),"1등 시상금: 3,000,000원, 총 시상금: 1천만원이하",■ 공모 주제\n- 아이디어 기획 : 공공데이터를 활용한 창의적인 아이디어\n- 제...,■ 지원 자격\n■ 지원 방법\n■ 시상내역\n- 홈페이지(커뮤니티 - 문의하기)
7,[통계청] 2024년 통계데이터 활용대회,통계청,5월 8일(수) ~ 6월 21일(금),"1등 시상금: 5,000,000원, 총 시상금: 3천만원~1천만원",[통계청] 2024년 통계데이터 활용대회\n\n\n■ 공모주제\n\n'데이터 혁신 ...,[통계청] 2024년 통계데이터 활용대회\n■ 공모주제\n■ 실시기관\n주최/주관 ...
8,제12회 산업통상자원부 공공데이터 활용 아이디어 공모전,산업통상자원,4월 30일(화) ~ 7월 1일(월),"1등 시상금: 100,000,000원, 총 시상금: 5천만원이상",공공데이터를 활용한 신규 사업기회 및 일자리 창출을 위한\n\n“제12회 산업통상자...,- 아이디어 기획 : 산업부 및 산업부 산하 공공기관 데이터를 활용하여 창의적인 아...
9,[인디크래프트] 진짜_리얼_최종_마감 D-Day!,성남시,3월 29일(금) ~ 4월 22일(월),"1등 시상금: 20,000,000원, 총 시상금: 5천만원이상",안녕하세요! ✨2024 인디크래프트✨에서 우수 개발사 선발을 위한 모집이 오늘 오후...,: 2024년 3월 29일(금) ~ 4월 22일(월) 오후 6시까지\n- 총 3억원...


In [ ]:
results4.to_csv("contest_list_4.csv", encoding="utf-8")

### 데이터 합치기

In [ ]:
# 3. 데이터 합치기
# 각자가 수집한 데이터셋을 하나로 합칩니다.
df = pd.concat([results1, results2, results3, results4], ignore_index=True)
df

,공모전명,기관명,기간,상금,공모전 소개,요약
0,2024 월드퀀트 국제 퀀트 모의투자 대회(IQC),월드퀀트,3월 19일(화) ~ 5월 15일(수),"1등 시상금: 정보 없음, 총 시상금: 5천만원이상",2024 월드퀀트 국제 퀀트 모의투자(IQC) 대회\n\n\n\n○ 공모개요\n- ...,2024 월드퀀트 국제 퀀트 모의투자(IQC) 대회\n○ 공모개요\n- Intern...
1,제4회 ICT융‧복합 기반 치의학산업 사업화 전국 아이디어 경진대회,(재)부산테크노파크,2023년 10월 16일(월) ~ 2023년 12월 10일(일),"1등 시상금: 3,000,000원, 총 시상금: 1천만원이하",* 모집기간 연장 (~12.10)\n[(재)부산테크노파크] 제4회 ICT융‧복합 기...,"- 의료산업 종사, (예비) 창업자 및 대학(원)생 등 ICT 치의학의료기기 산업에..."
2,(추가모집) 2024 AI 헬스케어 아이디어톤 경진대회,강원지역혁신플랫폼,8월 8일(목) ~ 8월 11일(일),"1등 시상금: 1,500,000원, 총 시상금: 1천만원이하","● 참가 자격\n- 강원지역혁신플랫폼 참여대학 대학(원)생\n* 가톨릭관동대학교, ...",● 참가 자격\n● 대회 주제 : IT 기술과 강원도 특화 산업을 융합한 AI 헬스...
3,[영림원소프트랩] 2024년 춘계통합학술대회 대학생 ERP 아이디어 공모,(주)영림원소프트랩,4월 11일(목) ~ 5월 7일(화),"1등 시상금: 2,000,000원, 총 시상금: 정보 없음",(주)영림원소프트랩과 한국경영정보학회 공동 주관으로 2024년 춘계통합학술대회에서 ...,- 5/7(화) 오전 9시까지 아이디어 개요서 제출 마감\n(상세 예시 첨부 포스터...
4,2024 핵테온 세종 국제 사이버 보안 위크 안내,세종특별자치시,3월 25일(월) ~ 4월 21일(일),"1등 시상금: 10,000,000원, 총 시상금: 5천만원~3천만원",세종특별자치시에서는 대학생 사이버보안 역량 강화 및 인재 육성·발굴·지원을 위해 2...,「핵테온 세종(HackTheon* Sejong)」 국제 대학생 사이버보안 경진대회를...
...,...,...,...,...,...,...
96,2024년 충북 공공데이터 활용 창업경진대회 모집공고,충청북도,3월 25일(월) ~ 5월 24일(금),"1등 시상금: 5,000,000원, 총 시상금: 1천만원이하",2024년 충청북도 공공데이터 활용 창업경진대회\n\n모집기간\n2024.03.25...,공공데이터 활용 (1)아이디어 기획 / (2)제품 및 서비스 개발\n신청서 및 제출...
97,2024 블록체인 경진대회 「BEST Challenge」,과학기술정보통신부,6월 14일(금) ~ 7월 23일(화),"1등 시상금: 5,000,000원, 총 시상금: 3천만원~1천만원",■ 대회명 : 2024 블록체인 경진대회 「BEST Challenge」\n■ 주최/...,■ 신청기간 : 2024년 6월 14일(금) ~ 7월 23일(화)\n■ 대회분야 :...
98,에이스프로젝트 제2회 야구 칼럼 공모전,에이스프로젝트,3월 4일(월) ~ 3월 29일(금),"1등 시상금: 500,000원, 총 시상금: 1천만원이하",[공모 개요]\n1982년부터 시작해 지금까지 변함없이 사랑 받고 있는 국민 스포츠...,"스포츠 그 이상의 가치, 작은 인생이 녹아있는 야구.\n[공모 주제]\n-각 구단 ..."
99,창업경진대회 CITYPRENEURS ULSAN 2024 참여기업 모집,"울산광역시, UN",3월 22일(금) ~ 4월 12일(금),"1등 시상금: 정보 없음, 총 시상금: 정보 없음",울산-UN 공동주최 창업경진대회「CITYPRENEURS ULSAN 2024」참여기업...,- 예선심사발표 | 4월 19일\n- 액셀러레이션 (온라인 진행) | 4월 22일 ...


In [ ]:
df = pd.concat([results1, results2, results3, results4], ignore_index=True)
df

,공모전명,기관명,기간,상금,공모전 소개,요약
0,2024 월드퀀트 국제 퀀트 모의투자 대회(IQC),월드퀀트,3월 19일(화) ~ 5월 15일(수),"1등 시상금: 정보 없음, 총 시상금: 5천만원이상",2024 월드퀀트 국제 퀀트 모의투자(IQC) 대회\n\n\n\n○ 공모개요\n- ...,2024 월드퀀트 국제 퀀트 모의투자(IQC) 대회\n○ 공모개요\n- Intern...
1,제4회 ICT융‧복합 기반 치의학산업 사업화 전국 아이디어 경진대회,(재)부산테크노파크,2023년 10월 16일(월) ~ 2023년 12월 10일(일),"1등 시상금: 3,000,000원, 총 시상금: 1천만원이하",* 모집기간 연장 (~12.10)\n[(재)부산테크노파크] 제4회 ICT융‧복합 기...,"- 의료산업 종사, (예비) 창업자 및 대학(원)생 등 ICT 치의학의료기기 산업에..."
2,(추가모집) 2024 AI 헬스케어 아이디어톤 경진대회,강원지역혁신플랫폼,8월 8일(목) ~ 8월 11일(일),"1등 시상금: 1,500,000원, 총 시상금: 1천만원이하","● 참가 자격\n- 강원지역혁신플랫폼 참여대학 대학(원)생\n* 가톨릭관동대학교, ...",● 참가 자격\n● 대회 주제 : IT 기술과 강원도 특화 산업을 융합한 AI 헬스...
3,[영림원소프트랩] 2024년 춘계통합학술대회 대학생 ERP 아이디어 공모,(주)영림원소프트랩,4월 11일(목) ~ 5월 7일(화),"1등 시상금: 2,000,000원, 총 시상금: 정보 없음",(주)영림원소프트랩과 한국경영정보학회 공동 주관으로 2024년 춘계통합학술대회에서 ...,- 5/7(화) 오전 9시까지 아이디어 개요서 제출 마감\n(상세 예시 첨부 포스터...
4,2024 핵테온 세종 국제 사이버 보안 위크 안내,세종특별자치시,3월 25일(월) ~ 4월 21일(일),"1등 시상금: 10,000,000원, 총 시상금: 5천만원~3천만원",세종특별자치시에서는 대학생 사이버보안 역량 강화 및 인재 육성·발굴·지원을 위해 2...,「핵테온 세종(HackTheon* Sejong)」 국제 대학생 사이버보안 경진대회를...
...,...,...,...,...,...,...
96,2024년 충북 공공데이터 활용 창업경진대회 모집공고,충청북도,3월 25일(월) ~ 5월 24일(금),"1등 시상금: 5,000,000원, 총 시상금: 1천만원이하",2024년 충청북도 공공데이터 활용 창업경진대회\n\n모집기간\n2024.03.25...,공공데이터 활용 (1)아이디어 기획 / (2)제품 및 서비스 개발\n신청서 및 제출...
97,2024 블록체인 경진대회 「BEST Challenge」,과학기술정보통신부,6월 14일(금) ~ 7월 23일(화),"1등 시상금: 5,000,000원, 총 시상금: 3천만원~1천만원",■ 대회명 : 2024 블록체인 경진대회 「BEST Challenge」\n■ 주최/...,■ 신청기간 : 2024년 6월 14일(금) ~ 7월 23일(화)\n■ 대회분야 :...
98,에이스프로젝트 제2회 야구 칼럼 공모전,에이스프로젝트,3월 4일(월) ~ 3월 29일(금),"1등 시상금: 500,000원, 총 시상금: 1천만원이하",[공모 개요]\n1982년부터 시작해 지금까지 변함없이 사랑 받고 있는 국민 스포츠...,"스포츠 그 이상의 가치, 작은 인생이 녹아있는 야구.\n[공모 주제]\n-각 구단 ..."
99,창업경진대회 CITYPRENEURS ULSAN 2024 참여기업 모집,"울산광역시, UN",3월 22일(금) ~ 4월 12일(금),"1등 시상금: 정보 없음, 총 시상금: 정보 없음",울산-UN 공동주최 창업경진대회「CITYPRENEURS ULSAN 2024」참여기업...,- 예선심사발표 | 4월 19일\n- 액셀러레이션 (온라인 진행) | 4월 22일 ...


In [ ]:
# 4. 공모전 기간 데이터 전처리 후 csv 파일로 저장하기

# 1) '기간' 칼럼을 공모전 '시작 날짜', '종료 날짜'로 분리하기
df[['시작 날짜', '종료 날짜']] = df['기간'].str.split(' ~ ', expand=True)

# "기간" 컬럼 삭제
df = df.drop(columns=['기간'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   공모전명    101 non-null    object
 1   기관명     101 non-null    object
 2   상금      101 non-null    object
 3   공모전 소개  101 non-null    object
 4   요약      101 non-null    object
 5   시작 날짜   101 non-null    object
 6   종료 날짜   100 non-null    object
dtypes: object(7)
memory usage: 5.6+ KB


In [ ]:
# 종료 날짜 열에서 결측값이 있는 행 찾기
missing_dates = df[df['종료 날짜'].isnull()]

# 결측값이 있는 행 출력
print("종료 날짜에서 결측값이 있는 행:")
print(missing_dates)

종료 날짜에서 결측값이 있는 행:
   공모전명 기관명                           상금 공모전 소개  요약 시작 날짜 종료 날짜
44       없음  1등 시상금: 정보 없음, 총 시상금: 정보 없음     없음  없음    없음  None


In [ ]:
df.drop(index=missing_dates.index, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 0 to 100
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   공모전명    100 non-null    object
 1   기관명     100 non-null    object
 2   상금      100 non-null    object
 3   공모전 소개  100 non-null    object
 4   요약      100 non-null    object
 5   시작 날짜   100 non-null    object
 6   종료 날짜   100 non-null    object
dtypes: object(7)
memory usage: 6.2+ KB


In [ ]:
# 2) 날짜 형식 통일하기: 예. 2022-01-01
from datetime import datetime

#년도가 없는 공모전은 임의로 현재 년도를 넣습니다.
current_year = datetime.now().year

# 날짜 문자열을 변환하는 함수
def convert_date(date_str):
    # 요일 정보 제거
    date_str = date_str.split('(')[0].strip()

    # 년, 월, 일을 분리
    date_parts = date_str.split()
    if len(date_parts) == 3:
        year, month, day = date_parts
        year = int(year[:-1])  # '년' 제거
    else:
        year = current_year
        month, day = date_parts

    month = int(month[:-1])  # '월' 제거
    day = int(day[:-1])  # '일' 제거

    # 날짜 문자열을 datetime 객체로 변환
    return datetime(year, month, day).strftime('%Y-%m-%d')

df['시작 날짜'] = df['시작 날짜'].apply(convert_date)
df['종료 날짜'] = df['종료 날짜'].apply(convert_date)

df

,공모전명,기관명,상금,공모전 소개,요약,시작 날짜,종료 날짜
0,2024 월드퀀트 국제 퀀트 모의투자 대회(IQC),월드퀀트,"1등 시상금: 정보 없음, 총 시상금: 5천만원이상",2024 월드퀀트 국제 퀀트 모의투자(IQC) 대회\n\n\n\n○ 공모개요\n- ...,2024 월드퀀트 국제 퀀트 모의투자(IQC) 대회\n○ 공모개요\n- Intern...,2024-03-19,2024-05-15
1,제4회 ICT융‧복합 기반 치의학산업 사업화 전국 아이디어 경진대회,(재)부산테크노파크,"1등 시상금: 3,000,000원, 총 시상금: 1천만원이하",* 모집기간 연장 (~12.10)\n[(재)부산테크노파크] 제4회 ICT융‧복합 기...,"- 의료산업 종사, (예비) 창업자 및 대학(원)생 등 ICT 치의학의료기기 산업에...",2023-10-16,2023-12-10
2,(추가모집) 2024 AI 헬스케어 아이디어톤 경진대회,강원지역혁신플랫폼,"1등 시상금: 1,500,000원, 총 시상금: 1천만원이하","● 참가 자격\n- 강원지역혁신플랫폼 참여대학 대학(원)생\n* 가톨릭관동대학교, ...",● 참가 자격\n● 대회 주제 : IT 기술과 강원도 특화 산업을 융합한 AI 헬스...,2024-08-08,2024-08-11
3,[영림원소프트랩] 2024년 춘계통합학술대회 대학생 ERP 아이디어 공모,(주)영림원소프트랩,"1등 시상금: 2,000,000원, 총 시상금: 정보 없음",(주)영림원소프트랩과 한국경영정보학회 공동 주관으로 2024년 춘계통합학술대회에서 ...,- 5/7(화) 오전 9시까지 아이디어 개요서 제출 마감\n(상세 예시 첨부 포스터...,2024-04-11,2024-05-07
4,2024 핵테온 세종 국제 사이버 보안 위크 안내,세종특별자치시,"1등 시상금: 10,000,000원, 총 시상금: 5천만원~3천만원",세종특별자치시에서는 대학생 사이버보안 역량 강화 및 인재 육성·발굴·지원을 위해 2...,「핵테온 세종(HackTheon* Sejong)」 국제 대학생 사이버보안 경진대회를...,2024-03-25,2024-04-21
...,...,...,...,...,...,...,...
96,2024년 충북 공공데이터 활용 창업경진대회 모집공고,충청북도,"1등 시상금: 5,000,000원, 총 시상금: 1천만원이하",2024년 충청북도 공공데이터 활용 창업경진대회\n\n모집기간\n2024.03.25...,공공데이터 활용 (1)아이디어 기획 / (2)제품 및 서비스 개발\n신청서 및 제출...,2024-03-25,2024-05-24
97,2024 블록체인 경진대회 「BEST Challenge」,과학기술정보통신부,"1등 시상금: 5,000,000원, 총 시상금: 3천만원~1천만원",■ 대회명 : 2024 블록체인 경진대회 「BEST Challenge」\n■ 주최/...,■ 신청기간 : 2024년 6월 14일(금) ~ 7월 23일(화)\n■ 대회분야 :...,2024-06-14,2024-07-23
98,에이스프로젝트 제2회 야구 칼럼 공모전,에이스프로젝트,"1등 시상금: 500,000원, 총 시상금: 1천만원이하",[공모 개요]\n1982년부터 시작해 지금까지 변함없이 사랑 받고 있는 국민 스포츠...,"스포츠 그 이상의 가치, 작은 인생이 녹아있는 야구.\n[공모 주제]\n-각 구단 ...",2024-03-04,2024-03-29
99,창업경진대회 CITYPRENEURS ULSAN 2024 참여기업 모집,"울산광역시, UN","1등 시상금: 정보 없음, 총 시상금: 정보 없음",울산-UN 공동주최 창업경진대회「CITYPRENEURS ULSAN 2024」참여기업...,- 예선심사발표 | 4월 19일\n- 액셀러레이션 (온라인 진행) | 4월 22일 ...,2024-03-22,2024-04-12


In [ ]:
df.to_csv("df_final.csv", encoding="utf-8", index= False)

# 2. 데이터 전처리

In [ ]:
## 웹크롤링 후 '상금' 칼럼의 경우, 다음의 데이터들은 수작업 전처리가 필요합니다.

# 대상: '기타', '상금 구간(e.g. 3천만~1천만)', '문자열 섞임(e.g. 3천만)', '금액이 너무 작음(e.g. 2원, 1,000원, 10000원 등)'
# 수정 결과: 공모전 별 1등 상금 또는 0
# 방식: '공모전 소개' 칼럼 참고하여 '1등 상금'이 구체적으로 명시되어 있는지 확인하고 없으면 해당 공모전을 검색하여 값을 수정한다.

# 새로운 상금값 선택 시 규칙: 수정 결과
#1) 구체적인 ‘상금’이 아닌 경우(e.g. 인턴십 기회, 해외 탐방 등 활동혜택, 상품): 0
#2) 동아리/창업팀을 지원해주는 성격의 공모전인 경우: 검색 시 가장 큰 액수의 지원금
#3) ‘공모전 소개’ 칼럼에 ‘추후 공개’로 적혀있는 경우: 검색 시 1등 상금이 나오면 그 값으로, 아니면 0
#4) ‘공모전 소개’ 칼럼에 총 상금만 명시되어 있거나 상금이 안 적혀있는 경우: 검색 시 1등 상금 나오면 그 값으로, 아니면 0
#5) 참가 대상이 대학생 or 대학원생, 시민 or 기업인 경우: 전자에게 지급된 1등 상금
#6) 상금이 달러인 경우: 달러 금액*1300원
#7) 참여 부문이 여러 개인 경우: 아이디어 제안 부문 보다 ‘데이터분석 활용/AI 개발’ 부문의 1등 상금 우선

In [ ]:
import pandas as pd

# '상금' 칼럼 수작업한 CSV file 불러오기
file_path = 'df_final.csv'
df = pd.read_csv(file_path)

# '천만'을 '10000000'으로 변환
df['상금'] = df['상금'].str.replace('천만', '0000000', regex=False)

# '원' 제거 및 쉼표 제거
df['상금'] = df['상금'].str.replace('원', '', regex=False).str.replace(',', '')

# 전처리된 CSV file 저장
cleaned_file_path = 'df_final_cleaned.csv'
df.to_csv(cleaned_file_path, index=False)

In [ ]:
#환경변수 설정
import os
#API key 넣기
os.environ["OPENAI_API_KEY"] = "오픈 API 키입니다."

In [ ]:
# 크롤링한 데이터 불러오기
import pandas as pd
df = pd.read_csv("df_final_cleaned.csv")
df

,공모전명,기관명,상금,공모전 소개,요약,시작 날짜,종료 날짜
0,2024 월드퀀트 국제 퀀트 모의투자 대회(IQC),월드퀀트,1등 시상금: 정보 없음 총 시상금: 50000000이상,2024 월드퀀트 국제 퀀트 모의투자(IQC) 대회\n\n\n\n○ 공모개요\n- ...,2024 월드퀀트 국제 퀀트 모의투자(IQC) 대회\n○ 공모개요\n- Intern...,2024-03-19,2024-05-15
1,제4회 ICT융‧복합 기반 치의학산업 사업화 전국 아이디어 경진대회,(재)부산테크노파크,1등 시상금: 3000000 총 시상금: 10000000이하,* 모집기간 연장 (~12.10)\n[(재)부산테크노파크] 제4회 ICT융‧복합 기...,"- 의료산업 종사, (예비) 창업자 및 대학(원)생 등 ICT 치의학의료기기 산업에...",2023-10-16,2023-12-10
2,(추가모집) 2024 AI 헬스케어 아이디어톤 경진대회,강원지역혁신플랫폼,1등 시상금: 1500000 총 시상금: 10000000이하,"● 참가 자격\n- 강원지역혁신플랫폼 참여대학 대학(원)생\n* 가톨릭관동대학교, ...",● 참가 자격\n● 대회 주제 : IT 기술과 강원도 특화 산업을 융합한 AI 헬스...,2024-08-08,2024-08-11
3,[영림원소프트랩] 2024년 춘계통합학술대회 대학생 ERP 아이디어 공모,(주)영림원소프트랩,1등 시상금: 2000000 총 시상금: 정보 없음,(주)영림원소프트랩과 한국경영정보학회 공동 주관으로 2024년 춘계통합학술대회에서 ...,- 5/7(화) 오전 9시까지 아이디어 개요서 제출 마감\n(상세 예시 첨부 포스터...,2024-04-11,2024-05-07
4,2024 핵테온 세종 국제 사이버 보안 위크 안내,세종특별자치시,1등 시상금: 10000000 총 시상금: 50000000~30000000,세종특별자치시에서는 대학생 사이버보안 역량 강화 및 인재 육성·발굴·지원을 위해 2...,「핵테온 세종(HackTheon* Sejong)」 국제 대학생 사이버보안 경진대회를...,2024-03-25,2024-04-21
...,...,...,...,...,...,...,...
95,2024년 충북 공공데이터 활용 창업경진대회 모집공고,충청북도,1등 시상금: 5000000 총 시상금: 10000000이하,2024년 충청북도 공공데이터 활용 창업경진대회\n\n모집기간\n2024.03.25...,공공데이터 활용 (1)아이디어 기획 / (2)제품 및 서비스 개발\n신청서 및 제출...,2024-03-25,2024-05-24
96,2024 블록체인 경진대회 「BEST Challenge」,과학기술정보통신부,1등 시상금: 5000000 총 시상금: 30000000~10000000,■ 대회명 : 2024 블록체인 경진대회 「BEST Challenge」\n■ 주최/...,■ 신청기간 : 2024년 6월 14일(금) ~ 7월 23일(화)\n■ 대회분야 :...,2024-06-14,2024-07-23
97,에이스프로젝트 제2회 야구 칼럼 공모전,에이스프로젝트,1등 시상금: 500000 총 시상금: 10000000이하,[공모 개요]\n1982년부터 시작해 지금까지 변함없이 사랑 받고 있는 국민 스포츠...,"스포츠 그 이상의 가치, 작은 인생이 녹아있는 야구.\n[공모 주제]\n-각 구단 ...",2024-03-04,2024-03-29
98,창업경진대회 CITYPRENEURS ULSAN 2024 참여기업 모집,"울산광역시, UN",1등 시상금: 정보 없음 총 시상금: 정보 없음,울산-UN 공동주최 창업경진대회「CITYPRENEURS ULSAN 2024」참여기업...,- 예선심사발표 | 4월 19일\n- 액셀러레이션 (온라인 진행) | 4월 22일 ...,2024-03-22,2024-04-12


In [ ]:
df1 = df.copy() #복사본
df1['요약2'] = ''#초기화 값
df1['tag'] = ''#초기화 값
df1['난이도'] = ''#초기화 값
df1['기관종류'] = ''#초기화 값

In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI  # OpenAI 패키지 임포트
import time  # 시간 관련 함수 사용을 위해 임포트

client = OpenAI()  # OpenAI 클라이언트 인스턴스 생성

# 공모전명과 공모전 소개를 요약하여 '요약' 컬럼 생성
def make_new_summary(text1, text2):
    try:
        # OpenAI GPT-3.5-turbo 모델을 사용하여 요약 생성
        completion = client.chat.completions.create(
          model="gpt-4o-mini",  # 사용할 모델 설정
          messages=[
            {"role": "system", "content": "위의 ‘공모전 소개’, ‘공모전명’에 나온 내용들을 요약 2에 요약해서 써줘. 이때 포맷은 반드시 참가 대상|공모 주제|시상 내역|기타 세부사항 4가지로 나눠서 반드시 이 순서대로 출력해줘(예:참가자 누구나|AI를 이용한 이미지 생성 모델 관련 공모전|1등 개발사에 2천만원 상금|접수기간:2023.04.05~2024.05.23, 문의사항:5578th@gmail.com). 그리고 이 4가지 외에는 아무것도 츨력하지 마. 그리고 맨 앞에서 '참가 대상|공모 주제|시상 내역|기타 세부사항'이라고 절대로, 절대로 출력하지 말고 반드시 '참가 대상|공모 주제|시상 내역|기타 세부사항'를 생략해주고, 이중 일부만 출력하는 것도 완전히 금지야(예:'참가 대상|AI를 이용한 이미지 생성 모델 관련 공모전|1등 개발사에 2천만원 상금|접수기간:2023.04.05~2024.05.23, 문의사항:5578th@gmail.com'등과 같이 '참가 대상'이라는 단어를 포함시키는 것도 금지야.). 만약 정보가 없거나 부족한 경우에는 '정보 없거나 부족' 이라고 출력해줘. 4가지 항목중 1개만 모르겠는 경우 반드시 그 부분만 '정보 없거나 부족'이라고 써줘. 다시 강조하지만 '참가 대상'이라는 단어를 사용해선 안돼."},
            {"role": "user", "content": f'{text1}, {text2}'}  # 사용자 역할로 입력 텍스트 제공
          ]
        )
        data = completion.choices[0].message  # 첫 번째 응답의 메시지 객체 추출
        category = data.content  # 메시지 객체의 내용 추출

        # TODO: 프린트 설정
        # 주석 처리 X: loop 당 GPT가 생성한 raw 데이터 확인 가능
        # 주석 처리 O: loop 당 raw data 출력 생략
        print(data)  # 요약된 내용 출력

        return category  # 요약된 내용 반환

    except OpenAI.error.APIError as e:  # API 에러 처리
        print(f"OpenAI API returned an API Error: {e}")
        time.sleep(120)  # 에러 발생 시 120초 대기
    except OpenAI.error.APIConnectionError as e:  # API 연결 에러 처리
        print(f"Failed to connect to OpenAI API: {e}")
        time.sleep(120)  # 에러 발생 시 120초 대기
    except OpenAI.error.RateLimitError as e:  # API 요청 한도 초과 에러 처리
        print(f"OpenAI API request exceeded rate limit: {e}")
        time.sleep(120)  # 에러 발생 시 120초 대기


In [ ]:
# 새로운 열 추가하여 태깅
df1['요약2'] = df1.apply(lambda row: make_new_summary(row['공모전명'], row['공모전 소개']), axis=1)

ChatCompletionMessage(content='전국 대학(원)생|시장 데이터와 연산자를 이용하여 주식 포지션을 시뮬레이팅하는 Alpha를 개발|총 상금 $40만불 내 시상, STAGE 1: $1,500/1등팀, STAGE 2: $3,000/1등팀, STAGE 3: $20,000/1등팀|접수기간:2023.03.19~2023.05.15, 글로벌 결승전:2023.07.23~2023.09.중순, 문의사항: 월드퀀트 한국지사 1:1문의', refusal=None, role='assistant', function_call=None, tool_calls=None)
ChatCompletionMessage(content='의료산업 종사자, (예비) 창업자 및 대학(원)생|ICT 융‧복합 및 제품화 가능한 구강보건산업분야 사업화 아이디어|대상(1명) 부산광역시장상 300만원, 우수상(2명) 각 200만원, 장려상(2명) 각 100만원|접수기간:2023.10.16~2023.12.10, 문의사항:051-974-9075, 이메일: lsh4188@btp.or.kr', refusal=None, role='assistant', function_call=None, tool_calls=None)
ChatCompletionMessage(content='강원지역혁신플랫폼 참여대학 대학(원)생|IT 기술과 강원도 특화 산업을 융합한 AI 헬스케어 신산업 아이디어 제안|대상·최우수상 3개팀 및 시상품|접수기간: 8월 11일 (일)까지, 문의사항: 033-250-6890, risdata@kangwon.ac.kr', refusal=None, role='assistant', function_call=None, tool_calls=None)
ChatCompletionMessage(content='대학생|ERP를 활용한 경영개선 아이디어|대상 1팀 상장/상금 200만원, 최우수상 2팀 상장/상금 각 100만원, 장려상 3팀 상장/상금 각 50만원|접수기간:5/7까지 아이디어 개요서 제출, 본선 PT

In [ ]:
# 요약2 칼럼을 보고 태깅을 진행하는 함수
def make_tagging(text):
    try:
        completion = client.chat.completions.create(
          model="gpt-4o-mini",
          messages=[
            {"role": "system", "content":"공모전 소개 항목을 보고 태깅을 해줘. 태깅하는 항목은 다음 6가지 중에서 골라줘. 6가지는 다음과 같아. '소프트웨어/서비스','콘텐츠(게임, 미디어)','프로그래밍/모델링','해킹/보안','UI/UX/디자인','데이터분석/문제해결'. 이 6가지의 항목 외에는 절대로 입력하지 마. 그리고 항목이 중복될 때에도 1개 까지만 입력하고 2개 이상 입력하지 마. 그리고 정보가 부족해서 어느 항목인지 찾을 수 없으면 '정보 없음' 이라고 출력하는데,'정보 없음'은 다른 항목과 중복해서 출력하지 말고 '정보 없음 하나만 출력해.(예 :'AI, 문제해결'과 같은 포맷으로 입력해줘. 절대로 'AI,정보없음'의 사례와 같이 '정보없음' 항목을 다른 항목과 같이 말하지 마. 'AI, 소프트웨어, 금융IT'등과 같이 3개 이상의 항목을 동시에 말하지 말고 최대 2개의 항목까지만 중복해서 말해.)"},
            {"role": "user","content": text}
          ]
        )
        data = completion.choices[0].message    # content 객체만 추출
        category = data.content

        # TODO: 프린트 설정
        # 주석 처리 X: loop 당 GPT가 생성한 raw 데이터 확인 가능
        # 주석 처리 O: loop 당 raw data 출력 생략
        print(data)

        return category

    except OpenAI.error.APIError as e:
        print(f"OpenAI API returned an API Error: {e}")
        time.sleep(120)
    except OpenAI.error.APIConnectionError as e:
        print(f"Failed to connect to OpenAI API: {e}")
        time.sleep(120)
    except OpenAI.error.RateLimitError as e:
        print(f"OpenAI API request exceeded rate limit: {e}")
        time.sleep(120)

In [ ]:
# 요약을 이용하여 tag 열 추가하여 태깅
df1['tag'] = df1['공모전 소개'].apply(make_tagging)

ChatCompletionMessage(content='데이터분석/문제해결', refusal=None, role='assistant', function_call=None, tool_calls=None)
ChatCompletionMessage(content='정보 없음', refusal=None, role='assistant', function_call=None, tool_calls=None)
ChatCompletionMessage(content='AI, 소프트웨어/서비스', refusal=None, role='assistant', function_call=None, tool_calls=None)
ChatCompletionMessage(content='데이터분석/문제해결', refusal=None, role='assistant', function_call=None, tool_calls=None)
ChatCompletionMessage(content='해킹/보안', refusal=None, role='assistant', function_call=None, tool_calls=None)
ChatCompletionMessage(content='데이터분석/문제해결', refusal=None, role='assistant', function_call=None, tool_calls=None)
ChatCompletionMessage(content='해킹/보안', refusal=None, role='assistant', function_call=None, tool_calls=None)
ChatCompletionMessage(content='UI/UX/디자인', refusal=None, role='assistant', function_call=None, tool_calls=None)
ChatCompletionMessage(content='데이터분석/문제해결', refusal=None, role='assistant', function_call=None, tool_calls=No

In [ ]:
#replace를 통한 답변 깔끔하게 정리
#GPT가 원하지 않는 유형의 답변을 생성했을 경우
df1['tag'] = df1['tag'].replace({'- 블록체인 아이디어톤: 정보 없음\n- 블록체인 ESG 해커톤: 정보 없음':'소프트웨어', 'AI, 소프트웨어, 문제해결':'AI, 소프트웨어',
                                 '문제해결, 데이터 분석, 빅데이터':'문제해결, 데이터 분석','AI, 데이터 분석, 헬스케어':'AI, 헬스케어','AI, 소프트웨어, 정보 없음':'AI, 소프트웨어',
                                'AI, 소프트웨어, 문제해결, 데이터 분석, 정보 없음':'AI, 소프트웨어','AI, 문제해결, 정보 없음':'AI, 문제해결','AI, 문제해결, 정보 없음':'AI, 문제해결',
                                 'AI, 소프트웨어, 창업':'AI, 창업','AI, 데이터 분석, 앱개발':'AI, 앱개발','AI, 소프트웨어, 문제해결, 앱개발':'AI, 앱개발',
                                 '알고리즘 문제를 풀고 소스코드를 제출하는 프로그래밍 경진대회로, 주로 소프트웨어 및 문제해결 분야와 관련이 있습니다.\n\n소프트웨어, 문제해결':'소프트웨어, 문제해결',
                                 '자율주행, 정보 없음':'자율주행','- 문제해결\n- 창업\n- 데이터 분석':'창업, 데이터 분석','- 문제해결\n- 창업\n- 데이터 분석':'창업, 데이터 분석','AI, 소프트웨어, 게임':'AI, 게임','블록체인, 문제해결, 금융IT':'문제해결, 금융IT',
                                 'AI, 정보 없음':'AI','게임, 정보 없음':'게임','창업, 정보 없음':'창업'})
df1['tag'].value_counts()

,count
tag,
데이터분석/문제해결,28
정보 없음,17
"콘텐츠(게임, 미디어)",15
소프트웨어/서비스,9
UI/UX/디자인,8
프로그래밍/모델링,8
"AI, 소프트웨어/서비스",5
"AI, 데이터분석/문제해결",4
해킹/보안,3


In [ ]:
# 난이도 결정 함수
def make_level(text):
    try:
        completion = client.chat.completions.create(
          model="gpt-4o-mini",
          messages=[
            {"role": "system", "content": "각 공모전을 공모전의 난이도에 따라 ‘상’, ‘중’, ‘하’ 로 분류하고 싶어. 다른 말을 쓰지 말고 '상', '중', '하', '없음' 중 하나로만 말해줘. 태그를 분류한 것에 ‘창업’이 있으면 난이도를 좀 더 높게 해주고, 상금에 따라서 '상', '중', '하' 사이에서 난이도를 조절해줘. 이때, 1등 상금이나 총 상금이 높을수록 난이도를 높게 측정해줘.난이도를 측정하기 어려운 경우에는 ‘없음’이라고 출력해줘."},
            {"role": "user","content": text},
          ],
          temperature=0.5,
        )
        data = completion.choices[0].message.content
        #여기 messages부분이 text를 주고 받는 곳
        # system에게 content를 주어 text를 받는 형태

        # TODO: 프린트 설정
        # 주석 처리 X: loop 당 GPT가 생성한 raw 데이터 확인 가능
        # 주석 처리 O: loop 당 raw data 출력 생략

        return data
    except OpenAI.error.APIError as e:
        print(f"OpenAI API returned an API Error: {e}")
        time.sleep(120)
    except OpenAI.error.APIConnectionError as e:
        print(f"Failed to connect to OpenAI API: {e}")
        time.sleep(120)
    except OpenAI.error.RateLimitError as e:
        print(f"OpenAI API request exceeded rate limit: {e}")
        time.sleep(120)

In [ ]:
# 난이도 열에 태깅
for i in range(len(df1)):
    df1['난이도'][i] = make_level(df1['공모전 소개'][i])

In [ ]:
#replace를 통한 답변 깔끔하게 정리
#GPT가 원하지 않는 유형의 답변을 생성했을 경우
df1['난이도'] = df1['난이도'].replace({'난이도: 중': '중', '이 문서의 내용은 중 난이도에 해당합니다.': '중', '난이도: 하': '하','': '없음',
                                       '문장이 없기 때문에 난이도를 판단할 수 없습니다. 다른 문장이나 내용을 제공해주시면 더 정확한 난이도 판단을 도와드릴 수 있습니다.': '없음',
                                       '문장이 없어서 난이도를 판단할 수 없습니다. 다시 한 번 문장을 입력해주시겠어요?': '없음',
                                       '문장이 짧고 단순하여 난이도를 평가할 수 없습니다.': '없음',
                                       '죄송합니다, 문장이 없어서 난이도를 판단할 수 없습니다. 다시 한번 문장을 입력해주시겠어요?': '없음',
                                       '죄송합니다. 질문이 없으셔서 답변을 드릴 수가 없네요. 궁금한 것이 있으시면 언제든지 질문해주세요!': '없음'})
df1['난이도'].value_counts()

,count
난이도,
상,65
중,34
없음,1


In [ ]:
def make_gov(text):
    try:
        completion = client.chat.completions.create(
          model="gpt-4o-mini",
          messages=[
            {"role": "system", "content": "이 데이터의 '기관명'컬럼을 보고 공모전의 주체 기관이 사기업인지, 공공기관인지 판단해줘.이때, 인터넷을 검색해서 일일히 둘중 하나로 결정해줘.만약 사기업이면 '사기업' 으로 입력하고, 공공기관이면 '공공기관'으로 입력해줘. 이때, 인터넷을 찾아봐도 모르겠으면'기타'라고 입력해줘, "},
            {"role": "user","content": text},
          ],
          temperature=0.5,
        )
        data = completion.choices[0].message.content
        #여기 messages부분이 text를 주고 받는 곳
        # system에게 content를 주어 text를 받는 형태

        # TODO: 프린트 설정
        # 주석 처리 X: loop 당 GPT가 생성한 raw 데이터 확인 가능
        # 주석 처리 O: loop 당 raw data 출력 생략

        return data
    except OpenAI.error.APIError as e:
        print(f"OpenAI API returned an API Error: {e}")
        time.sleep(120)
    except OpenAI.error.APIConnectionError as e:
        print(f"Failed to connect to OpenAI API: {e}")
        time.sleep(120)
    except OpenAI.error.RateLimitError as e:
        print(f"OpenAI API request exceeded rate limit: {e}")
        time.sleep(120)

In [ ]:
# 기관 열에 태깅
for i in range(len(df1)):
    df1['기관종류'][i] = make_gov(df1['기관명'][i])#일단 gpt로부터 답을 받는다.

In [ ]:
#replace를 통한 답변 깔끔하게 정리
df1['기관종류'] = df1['기관종류'].replace({'기업': '사기업'})
df1['기관종류'] = df1['기관종류'].apply(lambda x: x if x in ['공공기관', '사기업'] else '기타')
df1['기관종류'].value_counts()

,count
기관종류,
공공기관,51
기타,26
사기업,23


In [ ]:
# 생성한 데이터를 넣은 데이터 프레임 csv파일로 저장
df1.to_csv("final.csv",index = False)

# 3. 추천시스템 구현

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# 1) api로 태깅까지 완료된 데이터셋을 불러오기
data = pd.read_csv("final.csv")

In [ ]:
# 2) '공모전 소개' 열에서 '없음' 또는 null인 행을 삭제하기
df = data[df['공모전 소개'].notnull() & (df['공모전 소개'] != '없음')]
df

,공모전명,기관명,상금,공모전 소개,요약,시작 날짜,종료 날짜,요약2,tag,난이도,기관종류
0,2024 월드퀀트 국제 퀀트 모의투자 대회(IQC),월드퀀트,1등 시상금: 정보 없음 총 시상금: 50000000이상,2024 월드퀀트 국제 퀀트 모의투자(IQC) 대회\n\n\n\n○ 공모개요\n- ...,2024 월드퀀트 국제 퀀트 모의투자(IQC) 대회\n○ 공모개요\n- Intern...,2024-03-19,2024-05-15,전국 대학(원)생|시장 데이터와 연산자를 이용하여 주식 포지션을 시뮬레이팅하는 Al...,데이터분석/문제해결,상,사기업
1,제4회 ICT융‧복합 기반 치의학산업 사업화 전국 아이디어 경진대회,(재)부산테크노파크,1등 시상금: 3000000 총 시상금: 10000000이하,* 모집기간 연장 (~12.10)\n[(재)부산테크노파크] 제4회 ICT융‧복합 기...,"- 의료산업 종사, (예비) 창업자 및 대학(원)생 등 ICT 치의학의료기기 산업에...",2023-10-16,2023-12-10,"의료산업 종사자, (예비) 창업자 및 대학(원)생|ICT 융‧복합 및 제품화 가능한...",정보 없음,상,공공기관
2,(추가모집) 2024 AI 헬스케어 아이디어톤 경진대회,강원지역혁신플랫폼,1등 시상금: 1500000 총 시상금: 10000000이하,"● 참가 자격\n- 강원지역혁신플랫폼 참여대학 대학(원)생\n* 가톨릭관동대학교, ...",● 참가 자격\n● 대회 주제 : IT 기술과 강원도 특화 산업을 융합한 AI 헬스...,2024-08-08,2024-08-11,강원지역혁신플랫폼 참여대학 대학(원)생|IT 기술과 강원도 특화 산업을 융합한 AI...,"AI, 소프트웨어/서비스",상,공공기관
3,[영림원소프트랩] 2024년 춘계통합학술대회 대학생 ERP 아이디어 공모,(주)영림원소프트랩,1등 시상금: 2000000 총 시상금: 정보 없음,(주)영림원소프트랩과 한국경영정보학회 공동 주관으로 2024년 춘계통합학술대회에서 ...,- 5/7(화) 오전 9시까지 아이디어 개요서 제출 마감\n(상세 예시 첨부 포스터...,2024-04-11,2024-05-07,"대학생|ERP를 활용한 경영개선 아이디어|대상 1팀 상장/상금 200만원, 최우수상...",데이터분석/문제해결,중,사기업
4,2024 핵테온 세종 국제 사이버 보안 위크 안내,세종특별자치시,1등 시상금: 10000000 총 시상금: 50000000~30000000,세종특별자치시에서는 대학생 사이버보안 역량 강화 및 인재 육성·발굴·지원을 위해 2...,「핵테온 세종(HackTheon* Sejong)」 국제 대학생 사이버보안 경진대회를...,2024-03-25,2024-04-21,정보 없거나 부족|2024년 핵테온 세종(HackTheon Sejong) 국제 대학...,해킹/보안,상,공공기관
...,...,...,...,...,...,...,...,...,...,...,...
95,2024년 충북 공공데이터 활용 창업경진대회 모집공고,충청북도,1등 시상금: 5000000 총 시상금: 10000000이하,2024년 충청북도 공공데이터 활용 창업경진대회\n\n모집기간\n2024.03.25...,공공데이터 활용 (1)아이디어 기획 / (2)제품 및 서비스 개발\n신청서 및 제출...,2024-03-25,2024-05-24,대한민국 국민 누구나(개인 또는 5인 이내 팀)|공공데이터 활용 (1)아이디어 기획...,데이터분석/문제해결,상,공공기관
96,2024 블록체인 경진대회 「BEST Challenge」,과학기술정보통신부,1등 시상금: 5000000 총 시상금: 30000000~10000000,■ 대회명 : 2024 블록체인 경진대회 「BEST Challenge」\n■ 주최/...,■ 신청기간 : 2024년 6월 14일(금) ~ 7월 23일(화)\n■ 대회분야 :...,2024-06-14,2024-07-23,블록체인에 관심 있는 누구나(개인 또는 5인 이내 팀구성)|블록체인 기술을 활용한 ...,"블록체인, 소프트웨어/서비스",상,공공기관
97,에이스프로젝트 제2회 야구 칼럼 공모전,에이스프로젝트,1등 시상금: 500000 총 시상금: 10000000이하,[공모 개요]\n1982년부터 시작해 지금까지 변함없이 사랑 받고 있는 국민 스포츠...,"스포츠 그 이상의 가치, 작은 인생이 녹아있는 야구.\n[공모 주제]\n-각 구단 ...",2024-03-04,2024-03-29,"고등학생, 대학생, 직장인, 일반인 등 야구를 사랑하는 분들|야구에 대한 다양한 이...",정보 없음,중,사기업
98,창업경진대회 CITYPRENEURS ULSAN 2024 참여기업 모집,"울산광역시, UN",1등 시상금: 정보 없음 총 시상금: 정보 없음,울산-UN 공동주최 창업경진대회「CITYPRENEURS ULSAN 2024」참여기업...,- 예선심사발표 | 4월 19일\n- 액셀러레이션 (온라인 진행) | 4월 22일 ...,2024-03-22,2024-04-12,임팩트 스타트업|지속가능발전목표(SDGs) 달성 혹은 도시 문제 해결 솔루션|상금 ...,데이터분석/문제해결,상,기타


In [ ]:
# '시작 날짜' 컬럼의 자료형을 datetime으로 바꾸기
df['시작 날짜'] = pd.to_datetime(df['시작 날짜'], errors='coerce')

In [ ]:
# '종료 날짜' 컬럼의 자료형을 datetime으로 바꾸기
df['종료 날짜'] = pd.to_datetime(df['종료 날짜'], errors='coerce')

In [ ]:
# 변경된 데이터프레임 저장
df.to_csv("processed_final.csv", index=False)

df.head()

,공모전명,기관명,상금,공모전 소개,요약,시작 날짜,종료 날짜,요약2,tag,난이도,기관종류
0,2024 월드퀀트 국제 퀀트 모의투자 대회(IQC),월드퀀트,1등 시상금: 정보 없음 총 시상금: 50000000이상,2024 월드퀀트 국제 퀀트 모의투자(IQC) 대회\n\n\n\n○ 공모개요\n- ...,2024 월드퀀트 국제 퀀트 모의투자(IQC) 대회\n○ 공모개요\n- Intern...,2024-03-19,2024-05-15,전국 대학(원)생|시장 데이터와 연산자를 이용하여 주식 포지션을 시뮬레이팅하는 Al...,데이터분석/문제해결,상,사기업
1,제4회 ICT융‧복합 기반 치의학산업 사업화 전국 아이디어 경진대회,(재)부산테크노파크,1등 시상금: 3000000 총 시상금: 10000000이하,* 모집기간 연장 (~12.10)\n[(재)부산테크노파크] 제4회 ICT융‧복합 기...,"- 의료산업 종사, (예비) 창업자 및 대학(원)생 등 ICT 치의학의료기기 산업에...",2023-10-16,2023-12-10,"의료산업 종사자, (예비) 창업자 및 대학(원)생|ICT 융‧복합 및 제품화 가능한...",정보 없음,상,공공기관
2,(추가모집) 2024 AI 헬스케어 아이디어톤 경진대회,강원지역혁신플랫폼,1등 시상금: 1500000 총 시상금: 10000000이하,"● 참가 자격\n- 강원지역혁신플랫폼 참여대학 대학(원)생\n* 가톨릭관동대학교, ...",● 참가 자격\n● 대회 주제 : IT 기술과 강원도 특화 산업을 융합한 AI 헬스...,2024-08-08,2024-08-11,강원지역혁신플랫폼 참여대학 대학(원)생|IT 기술과 강원도 특화 산업을 융합한 AI...,"AI, 소프트웨어/서비스",상,공공기관
3,[영림원소프트랩] 2024년 춘계통합학술대회 대학생 ERP 아이디어 공모,(주)영림원소프트랩,1등 시상금: 2000000 총 시상금: 정보 없음,(주)영림원소프트랩과 한국경영정보학회 공동 주관으로 2024년 춘계통합학술대회에서 ...,- 5/7(화) 오전 9시까지 아이디어 개요서 제출 마감\n(상세 예시 첨부 포스터...,2024-04-11,2024-05-07,"대학생|ERP를 활용한 경영개선 아이디어|대상 1팀 상장/상금 200만원, 최우수상...",데이터분석/문제해결,중,사기업
4,2024 핵테온 세종 국제 사이버 보안 위크 안내,세종특별자치시,1등 시상금: 10000000 총 시상금: 50000000~30000000,세종특별자치시에서는 대학생 사이버보안 역량 강화 및 인재 육성·발굴·지원을 위해 2...,「핵테온 세종(HackTheon* Sejong)」 국제 대학생 사이버보안 경진대회를...,2024-03-25,2024-04-21,정보 없거나 부족|2024년 핵테온 세종(HackTheon Sejong) 국제 대학...,해킹/보안,상,공공기관


In [ ]:
# 2. 자신의 원하는 조건을 가진 공모전을 추리는 함수 만들기

In [ ]:
# 1) 필터링 함수 정의
# 매개변수: df, tag, 기관종류, 난이도
# 기본값: None
# 각 매개변수에 입력한 값이 None이 아닌 경우, 각 컬럼의 값이 입력값과 일치하는 행만 선택하도록 함수 구성

def filter_contests(df, tag=None, 기관종류=None, 난이도=None):
    #원본을 바꾸지 않기 위해 df 복사
    filtered_df = pd.read_csv("processed_final.csv")
    # tag_input 필터링
    if tag is not None:
        filtered_df = filtered_df[filtered_df['tag'] == tag]

    # 기관종류 필터링
    if 기관종류_input is not None:
        filtered_df = filtered_df[filtered_df['기관종류'] == 기관종류]

    # 난이도 필터링
    if 난이도_input is not None:
        filtered_df = filtered_df[filtered_df['난이도'] == 난이도]

    # 상금에 따라 내림차순으로 정렬
    filtered_df = filtered_df.sort_values(by='상금', ascending=False)

    return filtered_df

In [ ]:
# input함수 사용하여 각 매개변수마다 입력값 받기
# 공모전 종류(tag): [소프트웨어/서비스, 콘텐츠(게임, 미디어), 프로그래밍/모델링, 해킹/보안, UI/UX/디자인, 데이터분석/문제해결] 중 선택
# 기관 종류: [공공기관/사기업/기타] 중 선택
# 난이도 : [상/중/하] 중 선택
# 유효한 선택지 리스트
valid_tags = ["소프트웨어/서비스", "콘텐츠(게임, 미디어)", "프로그래밍/모델링", "해킹/보안", "UI/UX/디자인", "데이터분석/문제해결"]
valid_기관종류 = ["공공기관", "사기업", "기타"]
valid_난이도 = ["상", "중", "하"]

# 올바른 입력이 들어올 때까지 반복
while True:
    tag_input = input(f"공모전 종류(tag): {valid_tags} 중 선택: ")
    if tag_input in valid_tags:
        break
    else:
        print("올바른 공모전 종류를 선택해주세요.")

while True:
    기관종류_input = input(f"기관 종류: {valid_기관종류} 중 선택: ")
    if 기관종류_input in valid_기관종류:
        break
    else:
        print("올바른 기관 종류를 선택해주세요.")

while True:
    난이도_input = input(f"난이도 : {valid_난이도} 중 선택: ")
    if 난이도_input in valid_난이도:
        break
    else:
        print("올바른 난이도를 선택해주세요.")


공모전 종류(tag): ['소프트웨어/서비스', '콘텐츠(게임, 미디어)', '프로그래밍/모델링', '해킹/보안', 'UI/UX/디자인', '데이터분석/문제해결'] 중 선택: 데이터분석/문제해결
기관 종류: ['공공기관', '사기업', '기타'] 중 선택: 공공기관
난이도 : ['상', '중', '하'] 중 선택: 하


In [ ]:
# 2) 필터링된 결과 출력
# 매개변수 별 기본값: input 통해 받은 입력값
# 조건에 맞는 공모전이 없으면 나오는 출력값: "조건에 맞는 공모전이 없습니다."
# 그 외에는 상위 10개의 필터링된 공모전 출력하기
result_df = filter_contests(df, tag=tag_input, 기관종류=기관종류_input, 난이도=난이도_input)

if result_df.empty:
    print("조건에 맞는 공모전이 없습니다.")
else:
    # 상위 10개의 필터링된 공모전 출력
    print(result_df.head(10))

result_df

조건에 맞는 공모전이 없습니다.


,공모전명,기관명,상금,공모전 소개,요약,시작 날짜,종료 날짜,요약2,tag,난이도,기관종류


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# 3. 필터링된 공모전을 입력값으로 하는 추천시스템 구현하기
#반드시 skeleton 코드에 맞춰서 작성하지 않아도 됨. 단, 작성한 코드에 대해 어떤 알고리즘으로 결과가 나오게 했는지 설명할 수 있어야함.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 데이터프레임을 불러옵니다.
df = pd.read_csv("processed_final.csv")


# 1) contents-based filtering
#TF-IDF 벡터화: 요약2 컬럼을 사용
df['요약2'] = df['요약2'].fillna('')

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['요약2'])

# 코사인 유사도 행렬 계산

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
# 2) 추천 함수 생성
# 매개변수: df, contest_name, 공모전 간 코사인 유사도 행렬
df = pd.read_csv("processed_final.csv")

def recommendations(contest_name, cosine_sim=cosine_sim, df=df):
    # 입력한 공모전 이름과 일치하는 '공모전명' 컬럼 내 인덱스를 idx에 할당
    idx = df[df['공모전명'] == contest_name].index[0]

    # 해당 공모전과 모든 공모전 간의 유사도 점수를 가져오기
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도 점수를 기준으로 공모전 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 3개의 공모전 점수 가져오기
    sim_scores = sim_scores[1:4]  # 첫 번째는 자기 자신이므로 제외함

    # 공모전 인덱스 가져오기
    contest_index = [i[0] for i in sim_scores]

    # 공모전 인덱스를 이용하여 '공모전명' 칼럼에서 상위 3개의 유사한 공모전을 찾고 유사도 점수 출력
    recommended_contests = df['공모전명'].iloc[contest_index].values
    similarity_scores =  [i[1] for i in sim_scores]

    # enumerate, range 등을 사용하여 추천 공모전 출력
    # 출력 예: 1. 2021 OCR 인공지능 학습데이터 해커톤 | Similarity Score: 0.41
    for i in range(len(recommended_contests)):
        print(f"{i + 1}. {recommended_contests[i]} | Similarity Score: {similarity_scores[i]:.2f}")

In [ ]:
# 3) 함수 테스트
# input 함수로 공모전명 입력 받기
contest_name_input = input("공모전명을 입력하세요: ")
print(f"\n'{contest_name_input}'에 대한 추천 결과:")

# 추천함수 실행
recommendations(contest_name_input, cosine_sim, df)

공모전명을 입력하세요: 스마트농업 AI 경진대회

'스마트농업 AI 경진대회'에 대한 추천 결과:
1. [농림축산식품부] 스마트농업 AI 경진대회 | Similarity Score: 0.31
2. 제 7회 핀테크 아이디어 공모전 | Similarity Score: 0.25
3. 제 19회 경기게임오디션 | Similarity Score: 0.22
